In [1]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# Keras
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

# Sk learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
import hdbscan
import numpy as np

import re
import string
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
stopWords = set(stopwords.words('english'))

# from gensim.models.doc2vec import Doc2Vec




Using TensorFlow backend.


In [49]:

import gensim
# google_model_path= 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
google_model_path = './GoogleNews-vectors-negative300.bin.gz'
# Load Google's pre-trained Word2Vec model.
# model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)  
model = gensim.models.KeyedVectors.load_word2vec_format(google_model_path, binary=True, limit=10 ** 5)

In [27]:
df_original = pd.read_csv('../jigsaw-toxic-comment-classification-challenge/train.csv')

In [28]:
df_original.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0


In [29]:
df=df_original.copy(deep = True)
df=df.drop(columns = 'id')

In [30]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [31]:
regex_queries = [
#                 '@([A-Za-z0-9_]+)', #Usernames for twitter
#                'rt\s:\s', #Retweets marker
                 '(https?:\s?\/\s?\/)(\s)?(www\.)?(\s?)(\w+\.)*([\w\-\s]+\/)*([\w-]+)\/?' # Hyperlinks
                ]

df['text'] = df['comment_text'].replace(regex_queries,'',regex = True)
# Remove symbols
df['text'].replace('[^\w\s]|_','', regex = True,inplace = True)



In [32]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['text_tokenized'] = df['text'].apply(lambda x: tokenization(x.lower()))
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,text,text_tokenized
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation\nWhy the edits made under my usern...,"[explanation, why, the, edits, made, under, my..."
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,Daww He matches this background colour Im seem...,"[daww, he, matches, this, background, colour, ..."
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,Hey man Im really not trying to edit war Its j...,"[hey, man, im, really, not, trying, to, edit, ..."
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,\nMore\nI cant make any real suggestions on im...,"[, more, i, cant, make, any, real, suggestions..."
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,You sir are my hero Any chance you remember wh...,"[you, sir, are, my, hero, any, chance, you, re..."


In [33]:

def remove_stopwords(text):
    text = [word for word in text if word not in stopWords]
    return text
    
df['text_nonstop'] = df['text_tokenized'].apply(lambda x: remove_stopwords(x))
df.head(10)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,text,text_tokenized,text_nonstop
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation\nWhy the edits made under my usern...,"[explanation, why, the, edits, made, under, my...","[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,Daww He matches this background colour Im seem...,"[daww, he, matches, this, background, colour, ...","[daww, matches, background, colour, im, seemin..."
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,Hey man Im really not trying to edit war Its j...,"[hey, man, im, really, not, trying, to, edit, ...","[hey, man, im, really, trying, edit, war, guy,..."
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,\nMore\nI cant make any real suggestions on im...,"[, more, i, cant, make, any, real, suggestions...","[, cant, make, real, suggestions, improvement,..."
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,You sir are my hero Any chance you remember wh...,"[you, sir, are, my, hero, any, chance, you, re...","[sir, hero, chance, remember, page, thats]"
5,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,\n\nCongratulations from me as well use the to...,"[, congratulations, from, me, as, well, use, t...","[, congratulations, well, use, tools, well, ta..."
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,"[cocksucker, before, you, piss, around, on, my...","[cocksucker, piss, around, work]"
7,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,Your vandalism to the Matt Shirvington article...,"[your, vandalism, to, the, matt, shirvington, ...","[vandalism, matt, shirvington, article, revert..."
8,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,Sorry if the word nonsense was offensive to yo...,"[sorry, if, the, word, nonsense, was, offensiv...","[sorry, word, nonsense, offensive, anyway, im,..."
9,alignment on this subject and which are contra...,0,0,0,0,0,0,alignment on this subject and which are contra...,"[alignment, on, this, subject, and, which, are...","[alignment, subject, contrary, dulithgow]"


In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text if len(word) < 30]  # the length restriction prevents  
                                                                # long strings from overflowing recursion used in stemmer
    return text

df['text_stemmed'] = df['text_nonstop'].apply(lambda x: stemming(x))
df.head()

In [54]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text if len(word) < 30] 
    return text

df['text_lemmed'] = df['text_nonstop'].apply(lambda x: lemmatizer(x))
df.head(20)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,text,text_tokenized,text_nonstop,text_stemmed,text_lemmed
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation\nWhy the edits made under my usern...,"[explanation, why, the, edits, made, under, my...","[explanation, edits, made, username, hardcore,...","[explan, edit, made, usernam, hardcor, metalli...","[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,Daww He matches this background colour Im seem...,"[daww, he, matches, this, background, colour, ...","[daww, matches, background, colour, im, seemin...","[daww, match, background, colour, im, seemingl...","[daww, match, background, colour, im, seemingl..."
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,Hey man Im really not trying to edit war Its j...,"[hey, man, im, really, not, trying, to, edit, ...","[hey, man, im, really, trying, edit, war, guy,...","[hey, man, im, realli, tri, edit, war, guy, co...","[hey, man, im, really, trying, edit, war, guy,..."
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,\nMore\nI cant make any real suggestions on im...,"[, more, i, cant, make, any, real, suggestions...","[, cant, make, real, suggestions, improvement,...","[, cant, make, real, suggest, improv, wonder, ...","[, cant, make, real, suggestion, improvement, ..."
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,You sir are my hero Any chance you remember wh...,"[you, sir, are, my, hero, any, chance, you, re...","[sir, hero, chance, remember, page, thats]","[sir, hero, chanc, rememb, page, that]","[sir, hero, chance, remember, page, thats]"
5,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,\n\nCongratulations from me as well use the to...,"[, congratulations, from, me, as, well, use, t...","[, congratulations, well, use, tools, well, ta...","[, congratul, well, use, tool, well, talk, ]","[, congratulation, well, use, tool, well, talk, ]"
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,"[cocksucker, before, you, piss, around, on, my...","[cocksucker, piss, around, work]","[cocksuck, piss, around, work]","[cocksucker, piss, around, work]"
7,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,Your vandalism to the Matt Shirvington article...,"[your, vandalism, to, the, matt, shirvington, ...","[vandalism, matt, shirvington, article, revert...","[vandal, matt, shirvington, articl, revert, pl...","[vandalism, matt, shirvington, article, revert..."
8,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,Sorry if the word nonsense was offensive to yo...,"[sorry, if, the, word, nonsense, was, offensiv...","[sorry, word, nonsense, offensive, anyway, im,...","[sorri, word, nonsens, offens, anyway, im, int...","[sorry, word, nonsense, offensive, anyway, im,..."
9,alignment on this subject and which are contra...,0,0,0,0,0,0,alignment on this subject and which are contra...,"[alignment, on, this, subject, and, which, are...","[alignment, subject, contrary, dulithgow]","[align, subject, contrari, dulithgow]","[alignment, subject, contrary, dulithgow]"


In [56]:
model.wv('test')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


TypeError: 'Word2VecKeyedVectors' object is not callable

In [55]:
def vectorize_words(text):
        text = [model.get_vector(word) for word in text]
        return text
df['text_vectorized'] = df['text_lemmed'].apply(lambda x: vectorize_words(x))

KeyError: "word 'metallica' not in vocabulary"

In [48]:
# def clean_text(text):
#     text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
#     text_rc = re.sub('[0-9]+', '', text_lc)
#     tokens = re.split('\W+', text_rc)    # tokenization
#     text = [ps.stem(word) for word in tokens if word not in stopWords and len(word) < 30]  # remove stopwords and stemming
#     return text
# countVectorizer = CountVectorizer(analyzer=clean_text) 
# countVector = countVectorizer.fit_transform(df['text'])
# print('{} Number of Tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())



AttributeError: values not found

In [44]:
y_train = df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values

In [ ]:

NN = Sequential()

NN.add(Dense(100,  input_dim = X_train.shape[1])) # need feature input dim (28x28) for first hidden layer
NN.add(Activation('relu'))

NN.add(Dense(20))
NN.add(Activation('relu'))

NN.add(Dense(10)) # note we would typically use higher dim than this for last hidden layer
NN.add(Activation('relu', name = '2D_layer')) # naming this layer so we can extract it later

# Output layer

NN.add(Dense(6))
NN.add(Activation('sigmoid'))

# NN.compile(loss='categorical_crossentropy', optimizer='adam')
# For multi label (Classes are not mutually exclusive)
NN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
NN.fit(X_train, y_train, epochs=20, batch_size=512, verbose=1) # track progress as we fit